#### include library code

In [ ]:
using Random

#### define agents

In [ ]:
mutable struct ComplexHuman
    # var :: type
    migrant :: Bool
    employed :: Bool
    industry :: Int
    origin :: Country
    residence :: Country
    contacts :: Vector{ComplexHuman}
end

#### define countries

In [ ]:
mutable struct Country
    # this rate w/ be used in migration decision
    migration_rate :: Float64
    job_market :: Int
end